# Topic Modeling and Natural Language Processing with Twitter Data

##  Jason Anastasopoulos
##  July 30, 2018
### Email: ljanastas@princeton.edu

The code below provides a brief introduction on acquiring Twitter data using the twitter API via Python. For this exercise I will be acquiring Donald Trump's tweets and will try to figure out what the topics his tweets are using the Latent Dirichlet Allocation  Topic Model.

In [1]:
import csv
import os,re,csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import twitter
import json

Here we enter our Twitter credentials. These can be acquired through 

In [8]:
api = twitter.Api(consumer_key='mkz0izzVKDRzkrR4GoyN9FStT',
                      consumer_secret='4A1YGFEixYmyUNf2idYC33GKCuFoyJkyKpQVXIXCpDedZe0nOt',
                      access_token_key='18249358-xZGyGz8sWmQ9oJ1TBsLKEczwtO24aJ0Q4waDbjxAd',
                      access_token_secret='uqH7cC5BLS65iuAEPEv4TXEtUZvFD80wH03xkqiB7SP7Y')

print api.VerifyCredentials()

{"created_at": "Fri Dec 19 19:38:51 +0000 2008", "description": "Microsoft Visiting Professor @PrincetonCITP\u25aa\ufe0fAssistant Professor @UGA_PA_Policy & Political Science \u25aa\ufe0f political economy\u25aa\ufe0f machine learning\u25aa\ufe0f causal inference", "favourites_count": 920, "followers_count": 526, "friends_count": 625, "geo_enabled": true, "id": 18249358, "id_str": "18249358", "lang": "en", "listed_count": 30, "location": "Princeton, NJ", "name": "Jason Anastasopoulos", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/18249358/1522256331", "profile_image_url": "http://pbs.twimg.com/profile_images/953308288044683264/pfRYpPeN_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/953308288044683264/pfRYpPeN_normal.jpg", "profile

Search the Twitter API using keywords.

In [10]:
search = api.GetSearch("russian bots") # Replace happy with your search
for tweet in search:
    print(tweet.id, tweet.text)
    
len(search)

(1023267659671109633, u'#Russian bots fill stadiums during the 2016 campaign... https://t.co/dDE6hWD64W')
(1021722109943447552, u'I am much more concerned about the Marxists teaching our kids to hate America than Russian bots spreading nonsense on social media')
(1021100243687944192, u'#WalkAway is Russian propaganda \n#WalkAway is Russian propaganda \n#WalkAway is Russian propaganda \n#WalkAway is Rus\u2026 https://t.co/LoBB9PzGpu')
(1024058880274440192, u'@PlumRemson Tweets about vaccines from Russian bots... The election is less than 100 days away. This is one of many\u2026 https://t.co/f2mLzfnwnB')
(1024058788066811905, u'RT @SpockResists: \u274c\u274c\u274c\u274c\u274c\u274c\u274c\u274c\u274c\u274c\nRussian bots and trolls. They are using \u274c to say they are shadowbanned \U0001f644\U0001f644 but we can all see their tweets, so\u2026')
(1024058635305938944, u'@TheriotChristy @GovMikeHuckabee Sad, I know.\nBut I lost a shitload of Russian bots a week ago.\U0001f61e')
(10240582316

15

The Python twitter library has a lot of cool functions that you can use and learn about through the help() function

In [11]:
help(api.GetUserTimeline)


Help on method GetUserTimeline in module twitter.api:

GetUserTimeline(self, user_id=None, screen_name=None, since_id=None, max_id=None, count=None, include_rts=True, trim_user=False, exclude_replies=False) method of twitter.api.Api instance
    Fetch the sequence of public Status messages for a single user.
    
    The twitter.Api instance must be authenticated if the user is private.
    
    Args:
      user_id (int, optional):
        Specifies the ID of the user for whom to return the
        user_timeline. Helpful for disambiguating when a valid user ID
        is also a valid screen name.
      screen_name (str, optional):
        Specifies the screen name of the user for whom to return the
        user_timeline. Helpful for disambiguating when a valid screen
        name is also a user ID.
      since_id (int, optional):
        Returns results with an ID greater than (that is, more recent
        than) the specified ID. There are limits to the number of
        Tweets which c

In [12]:
t = api.GetUserTimeline(screen_name="realDonaldTrump", count=5000)
tweets = [i.AsDict() for i in t]

tweettext = [i["text"] for i in tweets]

tweettext[2:4]

[u'...that Open Borders, large scale Crime, and abolishing ICE is good for them, we must get smart and finally do what\u2026 https://t.co/9pMVWYu2MR',
 u'Illegal immigration is a top National Security problem. After decades of playing games, with the whole World laughi\u2026 https://t.co/4PueWN0YWA']

Loads the text data which are all referenda in New Mexico from 2000-2014. Splits referenda using "####" demarcation, extracts each bill and places it into a list.

## Cleaning the text

In [13]:
######## So far so good not lets clean this up ###
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
texts = []

Here we are pre-processing the text by creating a tokenizer that splits the documents up into tokens (words or phrases), creating a dictionary of stop words and creating a "stemmer" which stems the words (ie removing "-ing" endings etc. We also remove extraneous "bill related" words such as "propXX_XXXX".

In [14]:
for i in tweettext:
    #print "Processing",i
    # clean and tokenize document string
    tokens = tokenizer.tokenize(i)
    # remove all numbers
    tokens = [x for x in tokens if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    # remove structural words
    tokens = [x for x in tokens if len(x) > 1]
    tokens = [x.lower() for x in tokens]
    tokens = [x for x in tokens if 'http' not in x]
    tokens = [x for x in tokens if x not in "_"]
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)

dictionaryall = corpora.Dictionary(texts)

corpusall = [dictionaryall.doc2bow(text) for text in texts]

texts[1]



[u'thank',
 u'randpaul',
 u'ye',
 u'futur',
 u'great',
 u'justic',
 u'suprem',
 u'court',
 u'brett',
 u'kavanaugh',
 u'vote',
 u'mean',
 u'co',
 u'lg6ixxncxu']

In [32]:
texts[4]

[u'depart',
 u'justic',
 u'urg',
 u'suprem',
 u'court',
 u'least',
 u'hear',
 u'driver',
 u'licens',
 u'case',
 u'illeg',
 u'immi',
 u'co',
 u'teayq9in3t']

In [13]:
tweettext[0]

u'.....They can help solve problems with North Korea, Syria, Ukraine, ISIS, Iran and even the coming Arms Race. Bush\u2026 https://t.co/hrZ6vrJjVC'

This code performs tokenization, stop word removal and number removal and places the corpora into a clean list that will be ready for analysis using the Latent Dirichlet Allocation. 

Notice that there are two sets of texts that are jointly modeled "commercetexts" which are the "business friendly" Chamber of Commerce bills and "nmtexts" which are the New Mexico state referenda.

## Estimating the Latent Dirichlet Allocation model

In [15]:
# generate LDA model
ldamodelall = gensim.models.ldamodel.LdaModel(corpusall, num_topics=7, id2word = dictionaryall, passes=20,
                                              minimum_probability=0)



The code above estimates a 5 topic topic model using Trump's tweets

In [16]:
# 5 topics from Trump's tweets derived from the top 5 words

print(ldamodelall.print_topics(num_topics=5, num_words=5))

[(3, u'0.050*co + 0.014*news + 0.012*rt + 0.012*fake + 0.012*russia'), (5, u'0.058*co + 0.012*meet + 0.011*great + 0.008*now + 0.007*union'), (4, u'0.043*co + 0.020*rt + 0.017*will + 0.014*realdonaldtrump + 0.014*state'), (2, u'0.046*co + 0.013*democrat + 0.013*want + 0.011*year + 0.011*border'), (1, u'0.028*co + 0.014*david + 0.011*great + 0.011*state + 0.011*win')]


Prints the first 5 topics from the full model.

- Topic 1: "great" + 0.012*"countri" + 0.012*"will" + 0.009*"meet"' - Diplomatic Relations
- Topic 2: "better" + 0.008*"collus" + 0.008*"today" + 0.006*"much"' -  Russia Special Investigation
- Topic 3: "job" + 0.012*"trade" + 0.011*"thank" + 0.008*"year"' - Economy
- Topic 4: 0.015*"great" + 0.011*"north" + 0.011*"korea" + 0.009*"just"' - North Korea
- Topic 5: 0.015*"will" + 0.011*"state" + 0.011*"foxnew" + 0.009*"just"' - Media

# Breakout Group Exercise

1. Using my Twitter credentials collect 5000 tweets from the New York Times (@nytimes) timeline using the GetUserTImeline() function and also collect 5000 tweets from the Fox News timeline (@FoxNews).

2. Pre-process these tweets and estimate TWO k = 5 topic topic models for the New York Times tweets and the Fox News Tweets. Label the topics.

3. How do topics differ between the Times and Fox News at least according to their twitter accounts?

In [ ]:
# Starter code

# Credentials to use
api = twitter.Api(consumer_key='mkz0izzVKDRzkrR4GoyN9FStT',
                      consumer_secret='4A1YGFEixYmyUNf2idYC33GKCuFoyJkyKpQVXIXCpDedZe0nOt',
                      access_token_key='18249358-xZGyGz8sWmQ9oJ1TBsLKEczwtO24aJ0Q4waDbjxAd',
                      access_token_secret='uqH7cC5BLS65iuAEPEv4TXEtUZvFD80wH03xkqiB7SP7Y')


